In [0]:
%pip install pandas

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file using pandas
file_path = "./dados_churn_limpos.csv"
df = pd.read_csv(file_path)
df.head()

7. Telefonia (PhoneService, MultipleLines)

In [0]:
sns.set_style("whitegrid")

#Churn por Telefone e Múltiplas Linhas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

#Impacto de ter Telefone
sns.barplot(ax=axes[0], x='PhoneService', y='Churn', data=df, errorbar=None, palette="Blues_d")
axes[0].set_title('Churn por Serviço de Telefone')
axes[0].set_ylabel('Taxa de Churn')
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%.2f')

#Impacto de Múltiplas Linhas (filtrado apenas quem tem telefone)
df_phone_users = df[df['PhoneService'] == 'Yes']
sns.barplot(ax=axes[1], x='MultipleLines', y='Churn', data=df_phone_users, errorbar=None, palette="Reds_d")
axes[1].set_title('Churn por Múltiplas Linhas (Apenas quem tem telefone)')
axes[1].set_ylabel('Taxa de Churn')
for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%.2f')

plt.tight_layout()
plt.show()

#cruzamento internet com telefone para achar quem só tem telefone
pivot_table = pd.crosstab(
    index=df['InternetService'], 
    columns=df['PhoneService'], 
    values=df['Churn'], 
    aggfunc='mean'
)

plt.figure(figsize=(8, 5))
sns.heatmap(pivot_table, annot=True, fmt='.1%', cmap='RdYlGn_r', cbar_kws={'label': 'Taxa de Churn'})
plt.title('Heatmap: Risco de Churn por Combo (Internet x Telefone)')
plt.show()

print("--- DIAGNÓSTICO DE TELEFONIA ---")

# serviço legado: tem telefone mas não tem internet
churn_legado = df[(df['PhoneService'] == 'Yes') & (df['InternetService'] == 'No')]['Churn'].mean()
churn_geral = df['Churn'].mean()

print(f"1. Clientes que só têm telefonia (Legado) estão abandonando mais?")
print(f"   - Taxa Média Global: {churn_geral:.1%}")
print(f"   - Taxa Clientes Legados: {churn_legado:.1%}")

if churn_legado < churn_geral:
    print("   -> RESPOSTA: NÃO. Eles são mais fiéis que a média (Risco Baixo).")
else:
    print("   -> RESPOSTA: SIM. O serviço legado está gerando churn (Risco Alto).")

print("\n2. Clientes com múltiplas linhas são mais fiéis?")
churn_single = df[df['MultipleLines'] == 'No']['Churn'].mean()
churn_multi = df[df['MultipleLines'] == 'Yes']['Churn'].mean()

print(f"   - Churn Linha Única: {churn_single:.1%}")
print(f"   - Churn Múltiplas Linhas: {churn_multi:.1%}")

diff = churn_single - churn_multi
if diff > 0.01: 
    print("   -> RESPOSTA: SIM. Ter mais linhas reduz levemente o risco de saída.")
elif diff < -0.01:
    print("   -> RESPOSTA: NÃO. Ter múltiplas linhas aumenta o risco (verificar preço).")
else:
    print("   -> RESPOSTA: INDIFERENTE. Múltiplas linhas não impactam a fidelidade.")